In [120]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import section basically stolen from templates
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,LSTM,Dropout,Embedding
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from sklearn.metrics import mean_squared_error

In [92]:
times = pd.read_csv("algsWithTimesExtra.csv",delimiter='\t',header=None)

In [93]:
def split(x):
    return x.split(' ')

In [94]:
allMoves = set()

for line in times[2].apply(split):
    for move in line:
        allMoves.add(move)

allMoves = list(allMoves)

moveMapping = {move:idx for idx,move in enumerate(allMoves)}
maxline = times[2].apply(split).apply(len).max()

In [126]:
def processAlgForML(x):
    ans = []
    for move in x:
        ans.append(moveMapping[move])
    while len(ans) < maxline:
        ans.append(len(moveMapping)+1)
    return ans


In [127]:
x = pd.DataFrame(times[2].apply(split).apply(processAlgForML).tolist())
x_train,x_test,y_train,y_test = train_test_split(x,times[1])

In [130]:
def inputSingleAlg(x):
    ans = []
    for move in x.split(' '):
        ans.append(moveMapping[move])
    while len(ans) < maxline:
        ans.append(len(moveMapping)+1)
    return pd.DataFrame([ans])

In [129]:
inputSingleAlg("R2 F D B2 L D R B L D' R2 D2 R' U2 R' D' R' U2 B2")

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,25,8,10,16,17,10,29,1,17,28,...,23,24,20,24,28,24,20,16,99,99


In [125]:
len(moveMapping)

35

In [135]:
# No hyper paramter tuning at all.
model = Sequential()
model.add(Embedding(len(moveMapping)+2, 64, input_length=maxline)) 
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 21, 64)            2368      
                                                                 
 lstm_21 (LSTM)              (None, 21, 64)            33024     
                                                                 
 lstm_22 (LSTM)              (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 47,841
Trainable params: 47,841
Non-trainable params: 0
_________________________________________________________________


In [100]:
# NN_model = Sequential()

# # The Input Layer :
# NN_model.add(Dense(128, kernel_initializer='normal',input_dim = maxline, activation='relu'))
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# NN_model.summary()
# # NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# # NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

In [136]:
model.fit(x_train, y_train, epochs=500, batch_size=32, validation_split = 0.2)

Epoch 1/500
8/8 [==============================] - 2s 50ms/step - loss: 0.8907 - val_loss: 0.9451
Epoch 2/500
8/8 [==============================] - 0s 19ms/step - loss: 0.7783 - val_loss: 0.7201
Epoch 3/500
8/8 [==============================] - 0s 17ms/step - loss: 0.4080 - val_loss: 0.2143
Epoch 4/500
8/8 [==============================] - 0s 17ms/step - loss: 0.1795 - val_loss: 0.2219
Epoch 5/500
8/8 [==============================] - 0s 18ms/step - loss: 0.1548 - val_loss: 0.1838
Epoch 6/500
8/8 [==============================] - 0s 17ms/step - loss: 0.1259 - val_loss: 0.1795
Epoch 7/500
8/8 [==============================] - 0s 17ms/step - loss: 0.1206 - val_loss: 0.1524
Epoch 8/500
8/8 [==============================] - 0s 17ms/step - loss: 0.0986 - val_loss: 0.1366
Epoch 9/500
8/8 [==============================] - 0s 17ms/step - loss: 0.0965 - val_loss: 0.1253
Epoch 10/500
8/8 [==============================] - 0s 18ms/step - loss: 0.0802 - val_loss: 0.0897
Epoch 11/500
8/8 [=

In [137]:
mean_squared_error(model.predict(x_test),y_test)

4/4 [==============================] - 0s 5ms/step


0.02736572863905062

In [138]:
for xt,yt in zip(model.predict(x_test),y_test):
    print(xt[0],yt)

4/4 [==============================] - 0s 4ms/step
0.9620359 0.98
0.65571344 0.65
1.0130925 0.98
0.83231354 0.96
0.9396909 0.95
0.8013079 0.84
0.53621125 0.56
0.9605049 0.91
0.85696745 0.79
0.938923 0.92
0.814461 0.72
1.0192145 0.97
0.7542268 0.64
0.9354408 0.89
0.89068925 0.98
0.32674718 0.33
0.84660006 0.93
0.751292 0.83
0.8751446 0.96
0.91530985 0.92
0.6917831 0.68
0.75342345 0.67
0.9720441 0.99
0.8364713 0.8
0.9410148 0.98
0.5850965 0.59
0.7406585 0.63
0.79603803 0.67
0.8041312 0.83
0.9267742 0.94
0.96275437 0.92
0.8978194 0.78
0.8773111 0.93
0.67410815 0.79
0.9238855 0.88
0.908491 0.88
0.62307554 0.75
0.859894 0.87
0.96216613 0.92
1.0539441 1.07
1.0299475 1.08
0.92717814 0.97
0.9454717 0.99
0.75088364 0.77
0.896951 0.98
0.8763039 0.94
0.94447005 0.93
3.5083063 4.47
0.70272774 0.73
0.6266368 0.53
0.8830613 0.99
0.9439922 0.85
0.8759584 0.85
0.97771186 0.96
1.0293658 0.91
0.9160373 0.82
3.0938952 4.19
0.959686 0.94
0.896453 0.99
0.6866103 0.67
0.9598051 0.95
0.7889077 0.96
0.9390869

In [139]:
model.predict(inputSingleAlg("R2 F D B2 L D R B L D' R2 D2 R' U2 R' D' R' U2 B2"))

1/1 [==============================] - 0s 26ms/step


array([[2.3915875]], dtype=float32)

In [140]:
model.predict(inputSingleAlg("R U R' U R U2 R'"))

1/1 [==============================] - 0s 18ms/step


array([[0.3424716]], dtype=float32)

In [141]:
model.predict(inputSingleAlg("F U R U' R' U R U2 R' U' R U R' F'")), model.predict(inputSingleAlg("R' U R U R' U R U' R2 D' R U' R' D R U' R")) # This one should actually be like 0.80

1/1 [==============================] - 0s 20ms/step


(array([[0.88126934]], dtype=float32), array([[0.9245973]], dtype=float32))